In [9]:
import numpy as np
from scipy.optimize import minimize, NonlinearConstraint
from scipy.interpolate import interp1d

def objective_function(alpha, y_s):
    return np.sum((y_s + 1/alpha)**2)

def constraint(alpha, epsi):
    return np.concatenate([
        -1/epsi - alpha,
        1/(1 - epsi) + alpha,
        np.diff(alpha),
        np.diff(np.diff(alpha))
    ])

def LS(w, y, epsi):
    w_unique = np.unique(w)
    n = len(w_unique)
    w_sorted = np.sort(w_unique)
    index = np.argsort(w)

    y_s = [y[i] for i in index]
    y_s = np.array(y_s, dtype=int)

    # Initial guess for alpha
    alpha0 = -np.ones(n)

    # Define the objective and constraints for scipy.optimize
    obj_func = lambda alpha: objective_function(alpha, y_s)
    nonlinear_constraint = NonlinearConstraint(lambda alpha: constraint(alpha, epsi), -np.inf, 0)

    # Minimize the objective function subject to constraints
    result = minimize(obj_func, alpha0, constraints=[nonlinear_constraint])

    phi = result.x
    F_hat = interp1d(w_sorted, 1+1/phi, kind='linear', fill_value='extrapolate')
    return F_hat
